**Setting up the environment**

In [1]:
%%capture
files = !ls
files = [f.split("  ") for f in files][0]

isFRIQML = 'fri_qml' in files
isFRIQMLPath = isFRIQML and "setup.py" in files

# Clone the entire repo. Only run once!
if not isFRIQML:
  !git clone -l -s https://github.com/znajob/fri_qml.git fri_qml

if not isFRIQMLPath:
  %cd fri_qml

!git pull
!pip install -e .

In [2]:
# MAIN IMPORTS
import pennylane as qml
from pennylane import numpy as np
from friqml.visualisation import plot_quantum_state, plot_histogram
from friqml.utils import eps, random_state_normalized, random_state_unnormalized
from functools import partial
from tqdm.notebook import tqdm
from friqml.utils import eps

When solving the exercises refer to the [PennyLane documentation](https://pennylane.readthedocs.io/en/stable/).

## Quantum protocols

### Exercise 1

Write a template `cU` that implements a controlled single qubit unitary operation by using only the `CNOT` and single unitary gates. The template should have four arguments `alpha`, `beta`, `gamma`, and `delta` which specify the unitary as $U=\exp(i\alpha)R_z(\beta)R_y(\gamma)R_z(\delta)$. Hint: use the decomposition $U = \exp(i\alpha)AXBXC$, where $ABC=\mathbb{I}_2$.

In [3]:
# DEVICE
dev = qml.device('default.qubit', wires=2, shots=None)

In [40]:
def cU(alpha, beta, gamma, delta, wires=[0, 1]):
    qml.RZ(beta, wires=wires[1])
    qml.RY(gamma/2., wires=wires[1])

    qml.CNOT(wires)

    qml.RY(-gamma/2, wires=wires[1])
    qml.RZ(-(delta+beta)/2, wires=wires[1])

    qml.CNOT(wires)

    qml.RZ((delta-beta)/2, wires=wires[1])
    qml.PhaseShift(alpha,wires=wires[0])

In [41]:
@qml.qnode(dev)
def circuit_0(x):
  cU(x[0],x[1],x[2],x[3])
  return qml.state()

@qml.qnode(dev)
def circuit_1(x):
  qml.PauliX(wires=0)
  cU(x[0],x[1],x[2],x[3],wires=[0,1])
  qml.PhaseShift(-x[0],wires=0)
  qml.RZ(-x[3],wires=1)
  qml.RY(-x[2],wires=1)
  qml.RZ(-x[1],wires=1)
  return qml.state()

In [42]:
print(np.isclose(circuit_0(np.random.rand(4)*np.pi*2)[0],1))
print(np.isclose(abs(circuit_1(np.random.rand(4)*np.pi*2)[2]),1))

True
True


### Exercise 2

Write a template `teleportation` that implements the one qubit teleportation protocol. The first qubit should be transferred to the third qubit. The second and third qubit are initially in the state $|00⟩$.

In [64]:
# DEVICE
dev = qml.device('default.qubit', wires=3, shots=None)

In [69]:
###
### YOUR CODE HERE
###

def teleportation(wires=[0,1,2]):
    w0 = wires[0]
    w1 = wires[1]
    w2 = wires[2]

    qml.Hadamard(wires=1)
    qml.CNOT(wires=[w1, w2])

    qml.CNOT(wires=[w0, w1])
    qml.Hadamard(wires=0)

    qml.CNOT(wires=[w1, w2])
    qml.CZ(wires=[w0, w2])

In [70]:
@qml.qnode(dev)
def circuit(x):
  qml.RY(x,wires=0)
  teleportation(wires=[0,1,2])
  qml.RY(-x,wires=2)
  qml.Hadamard(wires=0)
  qml.Hadamard(wires=1)
  return qml.state()

In [71]:
# TESTS
print(np.isclose(circuit(2*np.random.rand()*np.pi)[0],1))

True



## Quantum fourier transform

### Exercise 1

The quantum fourier transform acts on the amplitudes of basis vectors. To use it on a classical vector $x$ we first have to encode it as amplitudes of the basis states. Such encoding of a classical vector on the amplitudes of quantum states is called amplitude encoding. Implement a templet `amplitude_encoding` that receives a vector $x=(x_0,x_1,x_2,x_3)$ and writes the elements into the amplitudes of a two qubit state.

In [72]:
# DEVICE
dev = qml.device('default.qubit', wires=2, shots=None)

In [ ]:
def amplitude_encoding(x, wires=[0, 1]):
    # Normalize the vector to represent a quantum state
    y = x/np.linalg.norm(x)
    x0 = y[0]
    x1 = y[1]
    x2 = y[2]
    x3 = y[3]
    fi0 = np.arccos(np.sqrt(x0**2+x1**2))
    fi1 = np.arctan2(x1, x0)
    fi2 = np.arctan2(x3, x2)
    fi3 = (fi2-fi1)
    qml.RY(2*fi0, wires=wires[0])
    qml.RY(2*fi1, wires=wires[1])
    U = np.array([[np.cos(fi3), -np.sin(fi3)], [np.sin(fi3), np.cos(fi3)]])
    qml.ControlledQubitUnitary(U, control_wires=[wires[0]], wires=wires[1])

In [ ]:
@qml.qnode(dev)
def circuit(x):
  amplitude_encoding(x,wires=[0,1])
  return qml.state()

In [ ]:
# TESTS
x = (1,0,0,0)
print(np.isclose(np.linalg.norm(x-circuit(x)),0))
x = (0,1,0,0)
print(np.isclose(np.linalg.norm(x-circuit(x)),0))
x = (0,0,1,0)
print(np.isclose(np.linalg.norm(x-circuit(x)),0))
x = (0,0,0,1)
print(np.isclose(np.linalg.norm(x-circuit(x)),0))
for i in range(10):
  x = np.random.rand(4) - np.random.rand(4)
  x /= np.linalg.norm(x)
  print(np.isclose(np.linalg.norm(x-circuit(x)),0))

True
True
True
True
True
True
True
True
True
True
True
True
True
True


### Exercise 2
Write a template `qft` that implements a quantum fourier transform on two qubits.   

In [ ]:
import dimod

In [ ]:
def qft_rot(k):
    return 2*np.pi/2**k


def qft(wires=[0, 1]):
    qml.Hadamard(wires=wires[0])
    qml.ControlledPhaseShift(qft_rot(2), wires=[wires[1], wires[0]])
    qml.Hadamard(wires=wires[1])
    qml.SWAP(wires=[wires[0], wires[1]])

In [ ]:
@qml.qnode(dev)
def circuit(x):
  amplitude_encoding(x,wires=[0,1])
  qft(wires=[0,1])
  return qml.state()


def DFT(x):
    """
    Simple implementation of the discrete Fourier Transform of a 1D real-valued vector x
    """

    N = len(x)
    n = np.arange(N)
    k = n.reshape((N, 1))
    e = np.exp(2j * np.pi * k * n / N)

    X = np.dot(e, x)/np.sqrt(N)

    return X

In [ ]:
# TESTS
x = (1,0,0,0)
print(np.isclose(np.linalg.norm(circuit(x)-DFT(x)),0))
x = (0,1,0,0)
print(np.isclose(np.linalg.norm(circuit(x)-DFT(x)),0))
x = (0,0,1,0)
print(np.isclose(np.linalg.norm(circuit(x)-DFT(x)),0))
x = (0,0,0,1)
print(np.isclose(np.linalg.norm(circuit(x)-DFT(x)),0))
for i in range(10):
  x = np.random.rand(4) - np.random.rand(4)
  x /= np.linalg.norm(x)
  print(np.isclose(np.linalg.norm(circuit(x)-DFT(x)),0))


True
True
True
True
True
True
True
True
True
True
True
True
True
True


### Exercise 3
Write a template `iqft` that implements the inverse quantum fourier transform on two qubits.

In [ ]:
def iqft(wires=[0, 1]):
    qml.SWAP(wires=[wires[0], wires[1]])
    qml.Hadamard(wires=wires[1])
    qml.ControlledPhaseShift(-qft_rot(2), wires=[wires[1], wires[0]])
    qml.Hadamard(wires=wires[0])

In [ ]:
@qml.qnode(dev)
def circuit(x):
  amplitude_encoding(x,wires=[0,1])
  qft(wires=[0,1])
  iqft(wires=[0,1])
  return qml.state()

In [ ]:
# TESTS
x = (1,0,0,0)
print(np.isclose(np.linalg.norm(x-circuit(x)),0))
x = (0,1,0,0)
print(np.isclose(np.linalg.norm(x-circuit(x)),0))
x = (0,0,1,0)
print(np.isclose(np.linalg.norm(x-circuit(x)),0))
x = (0,0,0,1)
print(np.isclose(np.linalg.norm(x-circuit(x)),0))
for i in range(10):
  x = np.random.rand(4) - np.random.rand(4)
  x /= np.linalg.norm(x)
  print(np.isclose(np.linalg.norm(x-circuit(x)),0))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
